Before starting, you will need to install some packages to reproduce the baseline.

In [1]:
!pip install tqdm
!pip install scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pathlib import Path
from tqdm import tqdm

import logging

import numpy as np
import pandas as pd

import torch

import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

In [3]:
# import data
PATH_COLAB = '/content/drive/MyDrive/challenge_ens_2023_small/moco_features.zip'
PATH_DEVICE = '..'
try:
    from google.colab import drive
    logging.info('Working on Colab.')
    
    # connect your drive to the session
    drive.mount('/content/drive')

    %cd /content/drive/MyDrive/challenge_data_ens_small/

    # unzip data into the colab session
    ! unzip $PATH_COLAB -d /content
    logging.info('Data unziped in your Drive.')

    %cd /content

    %cp -R drive/MyDrive/challenge_ens_2023_small/supplementary_data/ .
    %cp drive/MyDrive/challenge_ens_2023_small/train_output.csv .


except:
    logging.info('Working on your device.')
    
    data_exists = os.path.exists(PATH_DEVICE + '/train_input') and os.path.exists(PATH_DEVICE + '/test_input') and os.path.exists(PATH_DEVICE + '/train_output.csv')
    
    if data_exists:
        logging.info(f"Dataset found on device at : '{PATH_DEVICE}.'") 
    else:
        raise FileNotFoundError(f"Data folder not found at '{PATH_DEVICE}'")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/MyDrive/challenge_data_ens_small/'
/content
Archive:  /content/drive/MyDrive/challenge_ens_2023_small/moco_features.zip
replace /content/test_input/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
/content


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Data architecture

After downloading or unzipping the downloaded files, your data tree must have the following architecture in order to properly run the notebook:
```
your_data_dir/
├── train_output.csv
├── train_input/
│   ├── images/
│       ├── ID_001/
│           ├── ID_001_tile_000_17_170_43.jpg
...
│   └── moco_features/
│       ├── ID_001.npy
...
├── test_input/
│   ├── images/
│       ├── ID_003/
│           ├── ID_003_tile_000_16_114_93.jpg
...
│   └── moco_features/
│       ├── ID_003.npy
...
├── supplementary_data/
│   ├── baseline.ipynb
│   ├── test_metadata.csv
│   └── train_metadata.csv
```

For instance, `your_data_dir = /storage/DATA_CHALLENGE_ENS_2022/`


This notebook aims to reproduce the baseline method on this challenge called `MeanPool`. This method consists in a logistic regression learnt on top of tile-level MoCo V2 features averaged over the slides.

For a given slide $s$ with $N_s=1000$ tiles and corresponding MoCo V2 features $\mathbf{K_s} \in \mathbb{R}^{(1000,\,2048)}$, a slide-level average is performed over the tile axis.

For $j=1,...,2048$:

$$\overline{\mathbf{k_s}}(j) = \frac{1}{N_s} \sum_{i=1}^{N_s} \mathbf{K_s}(i, j) $$

Thus, the training input data for MeanPool consists of $S_{\text{train}}=344$ mean feature vectors $\mathbf{k_s}$, $s=1,...,S_{\text{train}}$, where $S_{\text{train}}$ denotes the number of training samples.

## Data loading

In [5]:
# put your own path to the data root directory (see example in `Data architecture` section)
data_dir = Path("./")

# load the training and testing data sets
train_features_dir = data_dir / "train_input" / "moco_features"
test_features_dir = data_dir / "test_input" / "moco_features"
df_train = pd.read_csv(data_dir  / "supplementary_data" / "train_metadata.csv")
df_test = pd.read_csv(data_dir  / "supplementary_data" / "test_metadata.csv")

# concatenate y_train and df_train
y_train = pd.read_csv(data_dir  / "train_output.csv")
df_train = df_train.merge(y_train, on="Sample ID")

print(f"Training data dimensions: {df_train.shape}")  # (344, 4)
df_train.head()

Training data dimensions: (344, 4)


,Sample ID,Patient ID,Center ID,Target
0,ID_001.npy,P_001,C_1,0
1,ID_002.npy,P_002,C_2,1
2,ID_005.npy,P_005,C_5,0
3,ID_006.npy,P_006,C_5,0
4,ID_007.npy,P_007,C_2,1


## Data processing

We now load the features matrices $\mathbf{K_s} \in \mathbb{R}^{(1000,\,2048)}$ for $s=1,...,344$ and perform slide-level averaging. This operation should take at most 5 minutes on your laptop.

In [6]:
#size_train = 50 #len(df_train)
#X_train = np.zeros((size_train, 1000, 2048))
#y_train = np.zeros((size_train), dtype=int)
centers_train = []
patients_train = []

for i, (sample, label, center, patient) in enumerate(tqdm(
    df_train[["Sample ID", "Target", "Center ID", "Patient ID"]].values
)):
    # load the coordinates and features (1000, 3+2048)
    #_features = np.load(train_features_dir / sample)
    # get coordinates (zoom level, tile x-coord on the slide, tile y-coord on the slide)
    # and the MoCo V2 features
    #coordinates, features = _features[:, :3], _features[:, 3:]  # Ks
    # slide-level averaging
    #X_train[i] = features
    #y_train[i] = label
    centers_train.append(center)
    patients_train.append(patient)
    

# convert to numpy arrays
#X_train = np.array(X_train)
#y_train = np.array(y_train)
centers_train = np.array(centers_train)
patients_train = np.array(patients_train)

100%|██████████| 344/344 [00:00<00:00, 545703.70it/s]


## Multilayer perceptron with a max over an image

In [87]:
from torch import nn
from torch.functional import F

class Perceptron(torch.nn.Module):
    def __init__(self):
        super(Perceptron, self).__init__()
        
        self.input = nn.Linear(2048,512)

        self.hidden = nn.Linear(512, 1)

        self.output = nn.Linear(1000, 1)

        self.dropout = torch.nn.Dropout(p = 0.1)

        self.relu = torch.nn.ReLU() # instead of Heaviside step fn  

        self.sigmoid = torch.nn.Sigmoid()

        self.max = torch.nn  
    
    def forward(self, x):

        x = self.input(x)
        x = self.relu(x)

        #x = self.dropout(x)
        
        x = self.hidden(x)
        x = self.sigmoid(x)
        
        x = x.squeeze()
        x = self.output(x)
        #x = self.sigmoid(x)

        
        #x = torch.max(x, dim=1).values # instead of Heaviside step fn

        #x = self.output(x)
        output = self.sigmoid(x)
        return output.squeeze()

In [124]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
    def __init__(self, df, features_dir, test = False):
        self.df = df
        self.features_dir = features_dir
        self.test = test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if self.test:
            sample = self.df.iloc[idx]["Sample ID"]
        else:
            sample, label = self.df.iloc[idx][["Sample ID", "Target"]]
            
        # load the coordinates and features (1000, 3+2048)
        _features = np.load(self.features_dir / sample)
        # get coordinates (zoom level, tile x-coord on the slide, tile y-coord on the slide)
        # and the MoCo V2 features
        coordinates, features = _features[:, :3], _features[:, 3:]  # Ks

        if self.test:
            return {'x': features}
        return {'x':features, 'y':label}

In [125]:
N_split = 250
train_dataset = MyDataset(df=df_train.iloc[:N_split], features_dir=train_features_dir)
val_dataset = MyDataset(df=df_train.iloc[N_split:], features_dir=train_features_dir)


In [126]:
from torch.utils.data import DataLoader


loader = DataLoader(dataset=train_dataset, batch_size=32)
val_loader = DataLoader(dataset=val_dataset, batch_size=32)



In [146]:
from torch.nn import BCELoss
from torch.optim import Adam

model = Perceptron().to(device)
criterion = BCELoss().to(device)
optimizer = Adam(model.parameters(), lr=0.01, weight_decay=1e-1)

In [155]:
class EarlyStopping:
    def __init__(self, tolerance=5, min_delta=0):

        self.tolerance = tolerance
        self.min_delta = min_delta
        self.counter = 0
        self.early_stop = False

    def __call__(self, train_loss, validation_loss):
        if (validation_loss - train_loss) > self.min_delta:
            self.counter +=1
            if self.counter >= self.tolerance:  
                self.early_stop = True
        elif train_loss > validation_loss:
            self.counter = 0

class SaveBestModel:
    """
    Class to save the best model while training. If the current epoch's 
    validation loss is less than the previous least less, then save the
    model state.
    """
    def __init__(
        self, best_valid_loss=float('inf'), save_path='outputs/best_model.pth'
    ):
        self.best_valid_loss = best_valid_loss
        self.save_path=save_path
        
    def __call__(
        self, current_valid_loss, 
        epoch, model, optimizer, criterion
    ):
        if current_valid_loss < self.best_valid_loss:
            self.best_valid_loss = current_valid_loss
            print(f"\nBest validation loss: {self.best_valid_loss}")
            print(f"\nSaving best model for epoch: {epoch+1}\n")
            torch.save({
                'epoch': epoch+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                }, self.save_path)

In [156]:
def train(loader, sigma = 1e-2):
    model.train()
    mean_loss = 0
    for i, val in enumerate(loader):
        labels = val['y'].to(device)
        x = val['x'].to(device)
        noise = sigma * torch.randn_like(x)

        x = x + noise

        optimizer.zero_grad()
        output = model(x)

        
        loss = criterion(output, labels.float())
        loss.backward()
        optimizer.step()
        mean_loss = 1/(i+1) * (loss - mean_loss) + i / (i+1) * mean_loss
    return mean_loss

In [161]:
def train_fold(model, optimizer, criterion, loader, val_loader=[], n_epoch = 10, verbose=1, sigma=1e-2, model_name='model'):
    early_stopping = EarlyStopping(tolerance = 5, min_delta=0.005)
    save_best_model = SaveBestModel(save_path='./outputs_'+model_name+'_best_weights.pth')
    
    for epoch in range(n_epoch):
        loss = train(loader, sigma=sigma)

        val_loss = 0
        for i, val in enumerate(val_loader):
            model.eval()

            labels = val['y'].to(device)
            x = val['x'].to(device)
            
            output = model(x)
            
            v_loss = criterion(output, labels.float())

            val_loss = 1/(i+1) * (v_loss - val_loss) + i / (i+1) * val_loss
        
        if verbose:
            print(f"Epoch {epoch} - loss {loss:.4f} - val loss {val_loss:.4f}")
        
        #best model
        save_best_model(val_loss, epoch=epoch, model=model, optimizer=optimizer, criterion=criterion)
        # early stopping
        early_stopping(loss, val_loss)
        if early_stopping.early_stop:
          print("We are at epoch:", epoch)
          break

In [162]:
train_fold(model, optimizer, criterion, loader, val_loader, n_epoch=300, verbose=1, sigma = 0.001)

Epoch 0 - loss 0.3444 - val loss 0.3436

Best validation loss: 0.3435823321342468

Saving best model for epoch: 1

Epoch 1 - loss 0.3522 - val loss 0.3298

Best validation loss: 0.3298187851905823

Saving best model for epoch: 2

Epoch 2 - loss 0.3311 - val loss 0.3400
Epoch 3 - loss 0.3526 - val loss 0.3335
Epoch 4 - loss 0.3242 - val loss 0.3290

Best validation loss: 0.32901954650878906

Saving best model for epoch: 5

Epoch 5 - loss 0.3425 - val loss 0.3366
Epoch 6 - loss 0.3471 - val loss 0.3534
Epoch 7 - loss 0.3332 - val loss 0.3502
Epoch 8 - loss 0.3488 - val loss 0.3362
Epoch 9 - loss 0.3239 - val loss 0.3346
Epoch 10 - loss 0.3390 - val loss 0.3334
Epoch 11 - loss 0.3517 - val loss 0.3512
Epoch 12 - loss 0.3328 - val loss 0.3531
Epoch 13 - loss 0.3403 - val loss 0.3478
Epoch 14 - loss 0.3211 - val loss 0.3441
Epoch 15 - loss 0.3445 - val loss 0.3343
Epoch 16 - loss 0.3222 - val loss 0.3440
Epoch 17 - loss 0.3465 - val loss 0.3327
Epoch 18 - loss 0.3226 - val loss 0.3529
Epoch

In [165]:
# load the best model checkpoint
best_model_cp = torch.load('outputs_model_best_weights.pth')
best_model_epoch = best_model_cp['epoch']
print(f"Best model was saved at {best_model_epoch} epochs\n")

Best model was saved at 5 epochs



In [167]:
best_model_cp = torch.load('outputs_model_best_weights.pth')
model.load_state_dict(best_model_cp['model_state_dict'])

<All keys matched successfully>

In [168]:
y_trues = []
preds = []
for val in val_loader:
    model.eval()
    y_true = val['y'].detach().numpy().tolist()
    x = val['x'].to(device)

    pred = model(x).detach().cpu().numpy().tolist()
    y_trues.extend(y_true)
    preds.extend(pred)

auc = roc_auc_score(y_trues, preds)

print(auc)

0.6618773946360154


## 5-fold cross validation

In [152]:
# /!\ we perform splits at the patient level so that all samples from the same patient
# are found in the same split

patients_unique = np.unique(patients_train)
y_unique = np.array(
    [np.mean(y_train[patients_train == p]) for p in patients_unique]
)
centers_unique = np.array(
    [centers_train[patients_train == p][0] for p in patients_unique]
)

print(
    "Training set specifications\n"
    "---------------------------\n"
    f"{len(df_train)} unique samples\n"
    f"{len(patients_unique)} unique patients\n"
    f"{len(np.unique(centers_unique))} unique centers"
)

Training set specifications
---------------------------
344 unique samples
305 unique patients
3 unique centers


/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3438: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [ ]:
aucs = []
models = []
# 5-fold CV is repeated 5 times with different random states
for k in range(5):
    kfold = StratifiedKFold(5, shuffle=True, random_state=k)
    fold = 0
    # split is performed at the patient-level
    for train_idx_, val_idx_ in kfold.split(patients_unique, y_unique):
        # retrieve the indexes of the samples corresponding to the
        # patients in `train_idx_` and `test_idx_`
        train_idx = np.arange(len(df_train))[
            pd.Series(patients_train).isin(patients_unique[train_idx_])
        ]
        val_idx = np.arange(len(df_train))[
            pd.Series(patients_train).isin(patients_unique[val_idx_])
        ]
        # set the training and validation folds
        df_fold_train = df_train.iloc[train_idx]
        data_fold_train = MyDataset(df_fold_train, train_features_dir)
        loader_fold_train = DataLoader(data_fold_train, batch_size=40, shuffle=True)

        df_fold_val = df_train.iloc[val_idx]
        data_fold_val = MyDataset(df_fold_val, train_features_dir)
        loader_fold_val = DataLoader(data_fold_val, batch_size=40, shuffle=True)
        
        # instantiate the model
        model = Perceptron().to(device)
        criterion = BCELoss().to(device)
        optimizer = Adam(model.parameters(), lr=0.01, weight_decay=1e-1)
        
        model_name = f'MLP_{k}_{fold}'
        train_fold(model, optimizer, criterion, loader_fold_train, val_loader=loader_fold_val, n_epoch=300, sigma=0.001, model_name=model_name)

        best_model_cp = torch.load('outputs_'+ model_name + '_best_weights.pth')
        model.load_state_dict(best_model_cp['model_state_dict'])
        
        # get the predictions (1-d probability)
        y_trues = []
        preds = []
        for val in loader_fold_val:
            model.eval()
            y_true = val['y'].detach().cpu().numpy().tolist()
            x = val['x'].to(device)

            pred = model(x).detach().cpu().numpy().tolist()
            y_trues.extend(y_true)
            preds.extend(pred)

        auc = roc_auc_score(y_trues, preds)

        print(f"AUC on split {k} fold {fold}: {auc:.3f}")
        aucs.append(auc)
        # add the logistic regression to the list of classifiers
        models.append(model)
        fold += 1
    print("----------------------------")
print(
    f"5-fold cross-validated AUC averaged over {k+1} repeats: "
    f"{np.mean(aucs):.3f} ({np.std(aucs):.3f})"
)

Epoch 0 - loss 0.4213 - val loss 0.3531

Best validation loss: 0.3531346917152405

Saving best model for epoch: 1

Epoch 1 - loss 0.3573 - val loss 0.3481

Best validation loss: 0.3480828106403351

Saving best model for epoch: 2

Epoch 2 - loss 0.3367 - val loss 0.3318

Best validation loss: 0.33182084560394287

Saving best model for epoch: 3

Epoch 3 - loss 0.3329 - val loss 0.3398
Epoch 4 - loss 0.3632 - val loss 0.3963
Epoch 5 - loss 0.3806 - val loss 0.3540
Epoch 6 - loss 0.3225 - val loss 0.3301

Best validation loss: 0.3301265239715576

Saving best model for epoch: 7

Epoch 7 - loss 0.3377 - val loss 0.3446
Epoch 8 - loss 0.3629 - val loss 0.3421
Epoch 9 - loss 0.3217 - val loss 0.3441
Epoch 10 - loss 0.3343 - val loss 0.3339
Epoch 11 - loss 0.3284 - val loss 0.3510
Epoch 12 - loss 0.3437 - val loss 0.3368
Epoch 13 - loss 0.3352 - val loss 0.3374
Epoch 14 - loss 0.3533 - val loss 0.3715
Epoch 15 - loss 0.3583 - val loss 0.3314
Epoch 16 - loss 0.3481 - val loss 0.3489
Epoch 17 - l

# Submission

Now we evaluate the previous models trained through cross-validation so that to produce a submission file that can directly be uploaded on the data challenge platform.

## Data processing

In [53]:
df_test

,Sample ID,Patient ID,Center ID
0,ID_003.npy,P_003,C_3
1,ID_004.npy,P_004,C_4
2,ID_008.npy,P_008,C_4
3,ID_009.npy,P_009,C_4
4,ID_010.npy,P_010,C_3
...,...,...,...
144,ID_482.npy,P_445,C_3
145,ID_487.npy,P_449,C_3
146,ID_488.npy,P_450,C_4
147,ID_492.npy,P_453,C_4


In [51]:
dataset_test = MyDataset(df_test, test_features_dir, test=True)

loader_test = DataLoader(dataset_test, batch_size=20, shuffle=False)

In [52]:
dataset_test[1]

FileNotFoundError: ignored

## Inference

In [44]:
preds_test = 0


# loop over the classifiers
for lr in models:
    
    preds = []
    for val in loader_test:
        model.eval()
        x = val['x'].to(device)

        pred = model(x).detach().numpy().tolist()
        preds.extend(pred)

    preds = np.array(preds)
    preds_test += preds
# and take the average (ensembling technique)
preds_test = preds_test / len(models)

TypeError: ignored

## Saving predictions

In [ ]:
submission = pd.DataFrame(
    {"Sample ID": df_test["Sample ID"].values, "Target": preds_test}
).sort_values(
    "Sample ID"
)  # extra step to sort the sample IDs

# sanity checks
assert all(submission["Target"].between(0, 1)), "`Target` values must be in [0, 1]"
assert submission.shape == (149, 2), "Your submission file must be of shape (149, 2)"
assert list(submission.columns) == [
    "Sample ID",
    "Target",
], "Your submission file must have columns `Sample ID` and `Target`"

# save the submission as a csv file
submission.to_csv(data_dir / "benchmark_test_output.csv", index=None)
submission.head()

# Dealing with images

The following code aims to load and manipulate the images provided as part of  this challenge.

## Scanning images paths on disk

This operation can take up to 5 minutes.

In [ ]:
train_images_dir = data_dir / "train_input" / "images"
train_images_files = list(train_images_dir.rglob("*.jpg"))

test_images_dir = data_dir / "test_input" / "images"
test_images_files = list(test_images_dir.rglob("*.jpg"))

print(
    f"Number of images\n"
    "-----------------\n"
    f"Train: {len(train_images_files)}\n" # 344 x 1000 = 344,000 tiles
    f"Test: {len(test_images_files)}\n"  # 149 x 1000 = 149,000 tiles
    f"Total: {len(train_images_files) + len(test_images_files)}\n"  # 493 x 1000 = 493,000 tiles
)

## Reading

Now we can load some of the `.jpg` images for a given sample, say `ID_001`.

In [ ]:
ID_001_tiles = [p for p in train_images_files if 'ID_001' in p.name]

In [ ]:
fig, axes = plt.subplots(5, 5)
fig.set_size_inches(12, 12)

for i, img_file in enumerate(ID_001_tiles[:25]):
    # get the metadata from the file path
    _, metadata = str(img_file).split("tile_")
    id_tile, level, x, y = metadata[:-4].split("_")
    img = plt.imread(img_file)
    ax = axes[i//5, i%5]
    ax.imshow(img)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f"Tile {id_tile} ({x}, {y})")
plt.show()

## Mapping with features

Note that the coordinates in the features matrices and tiles number are aligned.

In [ ]:
sample = "ID_001.npy"
_features = np.load(train_features_dir / sample)
coordinates, features = _features[:, :3], _features[:, 3:]
print("xy features coordinates")
coordinates[:10, 1:].astype(int)

In [ ]:
print(
    "Tiles numbering and features coordinates\n"
)
[tile.name for tile in ID_001_tiles[:10]]